In [1]:
import polars as pl
import altair as alt

In [2]:
df = pl.read_json("data_raw/papezstvo_raw_vatikan.json").with_columns(pl.col("vatikan_poradi").cast(int))

In [3]:
df

vatikan_poradi,vatikan_jmeno,vatikan_pontifikat_zacatek,vatikan_pontifikat_konec,vatikan_obcanske_jmeno,vatikan_narozeni_misto
i64,str,str,str,str,str
1,"""Peter""","""""","""64 or 67""","""""","""Bethsaida of Galilee"""
2,"""Linus""","""68""","""79""","""""","""Tuscia"""
3,"""Anacletus or Cletus""","""80""","""92""","""""","""Rome"""
4,"""Clement""","""92""","""99""","""""","""Rome"""
5,"""Evaristus""","""99 or 96""","""108""","""""","""Greece"""
…,…,…,…,…,…
263,"""John Paul I""","""26.VIII, 3.IX.1978""","""28.IX.1978""","""Albino Luciani""","""Forno di Canale (Belluno)"""
264,"""John Paul II""","""16,22.X.1978""","""2.IV.2005""","""Karol Wojtyła""","""Wadowice (Kraków)"""
265,"""Benedict XVI""","""19,24.IV.2005""","""28.II.2013""","""Joseph Ratzinger""","""Marktl am Inn (Bavaria)"""


In [4]:
def presnost(retezec):
    if " or " in retezec:
        if "." not in retezec:
            return 0
        else:
            return 1
    elif not "." in retezec:
        return 1
    elif "..." in retezec:
        return 1
    elif "." in retezec:
        return 2

In [5]:
print(presnost("...III.931"))
print(presnost("30.X,...XI.942"))
print(presnost("...XII.972, 19.I.973"))

1
1
1


In [6]:
df = df.with_columns(
    pl.col("vatikan_pontifikat_zacatek").map_elements(presnost).alias("vatikan_pontifikat_zacatek_presnost")
).with_columns(
    pl.col("vatikan_pontifikat_konec").map_elements(presnost).alias("vatikan_pontifikat_konec_presnost")
)

df

C:\Users\micha\AppData\Local\Temp\ipykernel_28332\2859633925.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(
C:\Users\micha\AppData\Local\Temp\ipykernel_28332\2859633925.py:3: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  ).with_columns(


vatikan_poradi,vatikan_jmeno,vatikan_pontifikat_zacatek,vatikan_pontifikat_konec,vatikan_obcanske_jmeno,vatikan_narozeni_misto,vatikan_pontifikat_zacatek_presnost,vatikan_pontifikat_konec_presnost
i64,str,str,str,str,str,i64,i64
1,"""Peter""","""""","""64 or 67""","""""","""Bethsaida of Galilee""",1,0
2,"""Linus""","""68""","""79""","""""","""Tuscia""",1,1
3,"""Anacletus or Cletus""","""80""","""92""","""""","""Rome""",1,1
4,"""Clement""","""92""","""99""","""""","""Rome""",1,1
5,"""Evaristus""","""99 or 96""","""108""","""""","""Greece""",0,1
…,…,…,…,…,…,…,…
263,"""John Paul I""","""26.VIII, 3.IX.1978""","""28.IX.1978""","""Albino Luciani""","""Forno di Canale (Belluno)""",2,2
264,"""John Paul II""","""16,22.X.1978""","""2.IV.2005""","""Karol Wojtyła""","""Wadowice (Kraków)""",2,2
265,"""Benedict XVI""","""19,24.IV.2005""","""28.II.2013""","""Joseph Ratzinger""","""Marktl am Inn (Bavaria)""",2,2


In [7]:
def ciste_datum(datum):

    # mega hnusné, spěchám
    
    rimske = ['I','II','III','IV','V','VI','VII','VIII','IX','X','XI','XII']
    rimske = [f'.{x}.' for x in rimske]
    rimske = dict(enumerate(rimske))
    rimske_ciste = {}
    for arabske, rimske in rimske.items():
        rimske_ciste[rimske] = f'.{arabske}.'
    if "," in datum:
        datum = datum.split(",")[1].strip()
    if " or " in datum:
        datum = datum.split(" or ")[1].strip()
    for rimske, arabske in rimske_ciste.items():
        datum = datum.replace(rimske, arabske)

    try:
        if '.' not in datum:
            datum = f'1.1.{datum}'
    except:
        return None
    
    return datum

In [8]:
ciste_datum("13,19.III.2013")

'19.2.2013'

In [9]:
ciste_datum("99 or 96"	)

'1.1.96'

In [10]:
df.filter(pl.col('vatikan_jmeno') == 'Benedict IX')

vatikan_poradi,vatikan_jmeno,vatikan_pontifikat_zacatek,vatikan_pontifikat_konec,vatikan_obcanske_jmeno,vatikan_narozeni_misto,vatikan_pontifikat_zacatek_presnost,vatikan_pontifikat_konec_presnost
i64,str,str,str,str,str,i64,i64
145,"""Benedict IX""","""...VIII or IX.1032""","""...IX.1044""","""Theophylactus of Tusculum""","""""",1,1
147,"""Benedict IX""","""10.III.1045""","""1.V.1045""","""Theophylactus of Tusculum""","""""",2,2
150,"""Benedict IX""","""...X.1047""","""... VIII.1048""","""Theophylactus of Tusculum""","""""",1,1


In [11]:
df

vatikan_poradi,vatikan_jmeno,vatikan_pontifikat_zacatek,vatikan_pontifikat_konec,vatikan_obcanske_jmeno,vatikan_narozeni_misto,vatikan_pontifikat_zacatek_presnost,vatikan_pontifikat_konec_presnost
i64,str,str,str,str,str,i64,i64
1,"""Peter""","""""","""64 or 67""","""""","""Bethsaida of Galilee""",1,0
2,"""Linus""","""68""","""79""","""""","""Tuscia""",1,1
3,"""Anacletus or Cletus""","""80""","""92""","""""","""Rome""",1,1
4,"""Clement""","""92""","""99""","""""","""Rome""",1,1
5,"""Evaristus""","""99 or 96""","""108""","""""","""Greece""",0,1
…,…,…,…,…,…,…,…
263,"""John Paul I""","""26.VIII, 3.IX.1978""","""28.IX.1978""","""Albino Luciani""","""Forno di Canale (Belluno)""",2,2
264,"""John Paul II""","""16,22.X.1978""","""2.IV.2005""","""Karol Wojtyła""","""Wadowice (Kraków)""",2,2
265,"""Benedict XVI""","""19,24.IV.2005""","""28.II.2013""","""Joseph Ratzinger""","""Marktl am Inn (Bavaria)""",2,2


In [12]:
df = df.with_columns(
    pl.col('vatikan_pontifikat_zacatek').map_elements(ciste_datum).str.to_date(format="%d.%m.%Y",strict=False)
).with_columns(
    pl.col('vatikan_pontifikat_konec').map_elements(ciste_datum).str.to_date(format="%d.%m.%Y",strict=False)
)

df

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


vatikan_poradi,vatikan_jmeno,vatikan_pontifikat_zacatek,vatikan_pontifikat_konec,vatikan_obcanske_jmeno,vatikan_narozeni_misto,vatikan_pontifikat_zacatek_presnost,vatikan_pontifikat_konec_presnost
i64,str,date,date,str,str,i64,i64
1,"""Peter""",null,0067-01-01,"""""","""Bethsaida of Galilee""",1,0
2,"""Linus""",0068-01-01,0079-01-01,"""""","""Tuscia""",1,1
3,"""Anacletus or Cletus""",0080-01-01,0092-01-01,"""""","""Rome""",1,1
4,"""Clement""",0092-01-01,0099-01-01,"""""","""Rome""",1,1
5,"""Evaristus""",0096-01-01,0108-01-01,"""""","""Greece""",0,1
…,…,…,…,…,…,…,…
263,"""John Paul I""",1978-08-03,1978-08-28,"""Albino Luciani""","""Forno di Canale (Belluno)""",2,2
264,"""John Paul II""",1978-09-22,2005-03-02,"""Karol Wojtyła""","""Wadowice (Kraków)""",2,2
265,"""Benedict XVI""",2005-03-24,2013-01-28,"""Joseph Ratzinger""","""Marktl am Inn (Bavaria)""",2,2


In [13]:
df.filter(pl.col('vatikan_jmeno') == 'Benedict IX')

vatikan_poradi,vatikan_jmeno,vatikan_pontifikat_zacatek,vatikan_pontifikat_konec,vatikan_obcanske_jmeno,vatikan_narozeni_misto,vatikan_pontifikat_zacatek_presnost,vatikan_pontifikat_konec_presnost
i64,str,date,date,str,str,i64,i64
145,"""Benedict IX""",null,null,"""Theophylactus of Tusculum""","""""",1,1
147,"""Benedict IX""",1045-02-10,1045-04-01,"""Theophylactus of Tusculum""","""""",2,2
150,"""Benedict IX""",null,null,"""Theophylactus of Tusculum""","""""",1,1


In [14]:
df.filter(pl.col('vatikan_pontifikat_zacatek').is_null())

vatikan_poradi,vatikan_jmeno,vatikan_pontifikat_zacatek,vatikan_pontifikat_konec,vatikan_obcanske_jmeno,vatikan_narozeni_misto,vatikan_pontifikat_zacatek_presnost,vatikan_pontifikat_konec_presnost
i64,str,date,date,str,str,i64,i64
1,"""Peter""",null,0067-01-01,"""""","""Bethsaida of Galilee""",1,0
20,"""Fabian""",null,null,"""""","""Rome""",1,2
22,"""Lucius I""",null,0254-02-05,"""""","""Rome""",1,2
26,"""Felix I""",null,0274-11-30,"""""","""Rome""",2,2
27,"""Eutichianus""",null,0283-11-07,"""""","""Luni""",2,2
…,…,…,…,…,…,…,…
201,"""Gregory XI""",null,1378-02-26,"""Pierre Roger de Beaufort""","""Rosiers d'Egletons (Limousin)""",2,2
224,"""Pius IV""",null,1565-11-09,"""Giovan Angelo Medici""","""Milan""",2,2
225,"""Pius V""",null,1572-04-01,"""Antonio (Michele) Ghisleri""","""Bosco (Alessandria)""",2,2


In [15]:
df.write_json("data_raw/papezstvo_raw_vatikan2.json")